<a href="https://colab.research.google.com/github/szambetti/INVEST_IO/blob/master/market_screener_reccom_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import json
import pprint
import numpy as np

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (126 kB/s)
Reading package lists... Done
Reading package lists... Done
Building de

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: use options instead of chrome_options


In [67]:
# RACCOMANDAZIONI
url = 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news-broker-research/'
wd.set_window_size(1920,1080)
wd.get(url)

# fai uno screenshot iniziale
wd.save_screenshot('initial_screenshot.png')

output = []

#loop sulle raccomandazioni dalla pagina principale
for i in wd.find_elements_by_css_selector('table#ALNI4.tabBody >* tr'):
  date = i.find_elements_by_css_selector('.pleft5.pright5.large30')[0].text
  reccom = i.find_elements_by_css_selector('.newsColCT.ptop3')[0].text
  link = i.find_elements_by_css_selector('.newsColCT.ptop3 > a')[0].get_attribute('href')
  output.append({'date': date, 'reccom' : reccom , 'link' : link})

In [68]:
output

[{'date': '02:29a',
  'link': 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news/INTESA-SANPAOLO-S-P-A-Upgraded-to-Buy-by-UBS-31076623/',
  'reccom': 'INTESA SANPAOLO S.P.A. : Upgraded to Buy by UBS'},
 {'date': '08/05',
  'link': 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news/INTESA-SANPAOLO-S-P-A-Kepler-Cheuvreux-reaffirms-its-Buy-rating-31061006/',
  'reccom': 'INTESA SANPAOLO S.P.A. : Kepler Cheuvreux reaffirms its Buy rating'},
 {'date': '08/04',
  'link': 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news/INTESA-SANPAOLO-S-P-A-Jefferies-reiterates-its-Buy-rating-31056906/',
  'reccom': 'INTESA SANPAOLO S.P.A. : Jefferies reiterates its Buy rating'},
 {'date': '08/04',
  'link': 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news/INTESA-SANPAOLO-S-P-A-Receives-a-Buy-rating-from-Jefferies-31053715/',
  'reccom': 'INTESA SANPAOLO S.P.A. : Receives a Buy rating from Jefferies'},
 {'date': '08/04',
  'link': 'https://www.marketscreene

In [69]:
def get_tds():
  wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  #loop sulle raccomandazioni dalla pagina principale
  for i in wd.find_elements_by_css_selector('table#ALNI4.tabBody >* tr'):
    date = i.find_elements_by_css_selector('.pleft5.pright5.large30')[0].text
    reccom = i.find_elements_by_css_selector('.newsColCT.ptop3')[0].text
    link = i.find_elements_by_css_selector('.newsColCT.ptop3 > a')[0].get_attribute('href')
    output.append({'date': date, 'reccom' : reccom , 'link' : link})

In [70]:
def click_on_next():
  element = wd.find_element_by_css_selector('a.nPageEndTab')
  wd.execute_script("arguments[0].click();", element)

In [71]:
#MAIN
url = 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news-broker-research/'
wd.set_window_size(1920,1080)
wd.maximize_window()
wd.get(url)

output = []
next_button = 1

while next_button > 0:
  get_tds()
  try:
    wd.find_element_by_css_selector('a.nPageEndTab')
    click_on_next()
  except:
    next_button = 0

output

[{'date': '02:29a',
  'link': 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news/INTESA-SANPAOLO-S-P-A-Upgraded-to-Buy-by-UBS-31076623/',
  'reccom': 'INTESA SANPAOLO S.P.A. : Upgraded to Buy by UBS'},
 {'date': '08/05',
  'link': 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news/INTESA-SANPAOLO-S-P-A-Kepler-Cheuvreux-reaffirms-its-Buy-rating-31061006/',
  'reccom': 'INTESA SANPAOLO S.P.A. : Kepler Cheuvreux reaffirms its Buy rating'},
 {'date': '08/04',
  'link': 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news/INTESA-SANPAOLO-S-P-A-Jefferies-reiterates-its-Buy-rating-31056906/',
  'reccom': 'INTESA SANPAOLO S.P.A. : Jefferies reiterates its Buy rating'},
 {'date': '08/04',
  'link': 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news/INTESA-SANPAOLO-S-P-A-Receives-a-Buy-rating-from-Jefferies-31053715/',
  'reccom': 'INTESA SANPAOLO S.P.A. : Receives a Buy rating from Jefferies'},
 {'date': '08/04',
  'link': 'https://www.marketscreene

In [73]:
full_output = []
import time
def loop_single_reccomm(k):
  url = k['link']
  date = k['date']
  title = k['reccom']
  wd.get(url)
  time.sleep(abs(np.random.normal(2,1)))
  time_published = wd.find_element_by_css_selector('meta[itemprop="datePublished"]').get_attribute('content')
  desc = wd.find_element_by_id('grantexto').text
  full_output.append({'date_short' : date, 'title' : title, 'url': url, 'date_long' : time_published, 'desc' : desc})

for i in output:
  loop_single_reccomm(i)
print (full_output)

[{'date_short': '02:29a', 'title': 'INTESA SANPAOLO S.P.A. : Upgraded to Buy by UBS', 'url': 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news/INTESA-SANPAOLO-S-P-A-Upgraded-to-Buy-by-UBS-31076623/', 'date_long': '2020-08-07T02:29:03-04:00', 'desc': "UBS's analyst Ignacio Cerezo upgrades his rating from Neutral to Buy. The target price has been lifted and is now set at EUR 2.40 compared to EUR 1.95 before."}]


In [ ]:
abs(np.random.normal(2,1))